In [21]:
import enviroment
import agents
import memory
import model
from tqdm import tqdm
from keras.models import Sequential , Model
from tensorflow.keras.layers import Concatenate, Input, Conv2D, Flatten, Dense
from keras.optimizers import Adam
from keras.initializers import RandomNormal
import policies
import time
import pandas as pd
import os

In [22]:
env = enviroment.SuperTicTacToe()

In [23]:
# Define the shape of the two inputs
input_shape1 = (1,) + env.observation_space.shape
input_shape2 = (1, 1)
# Define the two inputs
input1 = Input(shape=input_shape1)
input2 = Input(shape=input_shape2)
# First path
x = Conv2D(32, (3, 3), padding = 'same', activation='relu')(input1)  # Add a convolutional layer
# x = Conv2D(64, (3, 3), padding = 'same', activation='relu')(x)  # Add a convolutional layer
x = Flatten()(x)
# Concatenate the output of the first path with the second input
concatenated = Concatenate()([x, Flatten()(input2)])
# Add the final Dense layer
# output = Dense(16, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None))(concatenated)
# output = Dense(16, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None))(output)
output = Dense(9*9, kernel_initializer=RandomNormal(mean=0.0, stddev=0.05, seed=None))(concatenated)
# Create the model
net = Model(inputs=[input1, input2], outputs=output)
#compile the model
net.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

In [32]:
def get_experience(agent,iterations,mem_file,mdl_file,model_file):

    for i in tqdm(range(iterations)):

        agent.move(env)

        if i != 0:

            if i % 10:
                mdl.learn(32)
                mem.logging_stats()

            if i % 100 == 0:
                mdl.copy_weights()
            
            if i % 1000 == 0:

                episodes = agent.episodes

                #mem file and mdl file are in this form "name.format" i want to add the number of the iteration to the name in both
                mem_file_aux = mem_file.split(".")
                mem_file_aux = mem_file_aux[0] + str(episodes) + "." + mem_file_aux[1]
                mdl_file_aux = mdl_file.split(".")
                mdl_file_aux = mdl_file_aux[0] + str(episodes) + "." + mdl_file_aux[1]
                model_file_aux = model_file.split(".")
                model_file_aux = model_file_aux[0] + str(episodes) + "." + model_file_aux[1]

                #save the model and the memory in the folder
                mem.save(mem_file_aux)
                mdl.save(mdl_file_aux)
                agent.save(model_file_aux)
    
    mem.save(mem_file)
    mdl.save(mdl_file)
    agent.save(model_file)

In [33]:
folder = r'D:\berti\Documents\Machine Learning\SuperTicTacToe\SuperTicTacToeRL\resources\MinMaxAgents\1'

model_file = folder + r'\model_info.txt'
mem_file = folder + r'\memory.bin'
mdl_file = folder + r'\model.h5'
mem_log = folder + r'\memory_log.csv'

mem = memory.Memory(10000, mem_log)

policy = policies.Probabilistic(0.03)

mdl = model.Model(memory = mem, model = net, future = 0.68, alfa = 0.25)

dqn_agent = agents.dqn_agent(model = mdl, memory = mem, policy = policy)

#see if this files exist, if they do, load them
try:
    mem.load(mem_file)
except:
    print("Memory file not found")

try:
    mdl.load(mdl_file)
except:
    print("Model file not found")

try:
    dqn_agent.load(model_file)
except:
    print("Model file not found")

In [34]:
get_experience(dqn_agent, 10000, mem_file, mdl_file, model_file)

100%|██████████| 10000/10000 [1:09:34<00:00,  2.40it/s]


In [42]:
#try the agent

folder = r'D:\berti\Documents\Machine Learning\SuperTicTacToe\SuperTicTacToeRL\resources\heatmaps'

#delete all files in the folder
for file in os.listdir(folder):
    os.remove(folder + "\\" + file)

policy.exploration_rate = 0.03

while True:
    _,_,done,message = dqn_agent.move(env,render=True)
    print(message)
    if done:
        break

Correct move
Correct move
Correct move
Correct move
Correct move
Correct move
Correct move
Correct move
Correct move
Correct move
Correct move
Correct move
Invalid board
